In [1]:
from necro_uncal_new import model
from pylab import *
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
from pysb.util import alias_model_components
from pysb.simulator import CupSodaSimulator
from pysb.simulator import ScipyOdeSimulator
from pysb.simulator.bng import BngSimulator
# from necro import model
import pandas as pd
from pysb.simulator import SimulationResult
alias_model_components(model)

In [2]:
par_files = np.load('necro_pydream_5chns_929_alltnf.npy')
n_pars = len(par_files)
all_pars = np.zeros((n_pars, 54))

rate_params = model.parameters_rules() # these are only the parameters involved in the rules
param_values = np.array([p.value for p in model.parameters]) # these are all the parameters
rate_mask = np.array([p in rate_params for p in model.parameters])

for i in range(n_pars):
    par = par_files[i]
    param_values[rate_mask] = 10 ** par
    all_pars[i] = param_values
print(len(all_pars))

25000


In [3]:
# l = [i for i in range(10)]
flip = [3900, 1170]
tspan = np.linspace(0, 1440, 500)
solver100 = ScipyOdeSimulator(model, tspan=tspan, verbose = True)
result100 = solver100.run(param_values=all_pars[:5000],num_processors=20)
df = result100.dataframe

2019-10-22 21:36:31.465 - pysb.simulator.scipyode - DEBUG - [necro_uncal_new] Simulator created
2019-10-22 21:36:32.051 - pysb.simulator.scipyode - DEBUG - [necro_uncal_new] Equation mode set to "cython"
2019-10-22 21:36:32.109 - pysb.simulator.scipyode - INFO - [necro_uncal_new] Simulation(s) started
2019-10-22 21:36:33.045 - pysb.simulator.scipyode - DEBUG - [necro_uncal_new] Multi-processor (parallel) mode using 20 processes
2019-10-22 22:04:03.290 - pysb.simulator.scipyode - INFO - [necro_uncal_new] All simulation(s) complete
2019-10-22 22:04:03.294 - pysb.simulator.scipyode - DEBUG - [necro_uncal_new] SimulationResult constructor started
2019-10-22 22:04:05.707 - pysb.simulator.scipyode - DEBUG - [necro_uncal_new] SimulationResult constructor finished


In [ ]:
# tspan = np.linspace(0, 1440, 500)
x100 = np.array([30, 90, 270, 480, 600, 720, 840, 960])
y100 = np.array([0.00885691708746097,0.0161886154261265,0.0373005242261882,0.2798939020159581,0.510, .7797294067, 0.95,1])
plt.figure()
for i in range(1000):
# plt.subplot(144)
    plt.plot(tspan/60, df.loc[i]['MLKLa_obs'].iloc[:]/df.loc[i]['MLKLa_obs'].iloc[:].max(), zorder = 1)
# plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
plt.scatter(x100/60, y100, zorder = 2)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
# plt.legend()
plt.title(r'25000 fitted parameters of pMKLK under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_25000params_TNF1_pydream_5chns_95CI.pdf', format='pdf')
plt.show()

In [ ]:
df['MLKLa_obs'].std(axis = 1)

In [ ]:
mean_mlk = []
for i in range(10000):
    mean_mlkl[i] = np.std(df.loc[i]['MLKLa_obs'].iloc[i])
# print(mean_mlk)    
# sd_mlkl = df.loc[:]['MLKLa_obs'].iloc[:].std()

In [ ]:
std = np.zeros(10000)
for i in range (10000):
       std = df.loc[i]['MLKLa_obs'].iloc[i]

In [ ]:
sd_mlkl = df.loc[:]['MLKLa_obs'].iloc[:].std()
sd_mlkl

In [ ]:
mean_mlkl

In [ ]:
result100.save('testing_pydream_10000params.h5')

In [ ]:
result100 = SimulationResult.load('necro_25000params_100TNF_pydream_5chns_500steps.h5') #loading in h5 result file 
df = result100.dataframe #creating pandas dataframe 

In [ ]:
result100.obser

In [ ]:
mlkl = df.loc[:]['MLKLa_obs'].iloc[:]
mlkl.shape
len(mlkl)

In [ ]:
print(result100.observables)

In [ ]:
mean_pmlkl = df.loc[:]['MLKLa_obs'].iloc[:].values.reshape((100,))

In [ ]:
tspan = np.linspace(0, 1440, 500)
x100 = np.array([30, 90, 270, 480, 600, 720, 840, 960])
y100 = np.array([0.00885691708746097,0.0161886154261265,0.0373005242261882,0.2798939020159581,0.510, .7797294067, 0.95,1])
plt.figure()
for i in range(len(mlkl)):
# plt.subplot(144)
    plt.plot(tspan/60, df.loc[i]['MLKLa_obs'].iloc[:])
# plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
# plt.scatter(x100/60, y100)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
# plt.legend()
plt.title(r'95% CI of pMLKL under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_25000params_TNF1_pydream_5chns_95CI.pdf', format='pdf')
plt.show()

In [ ]:
m = model.monomers #to obtain model monomers and add on demand observables

In [ ]:
df['Flip_obs'] = result100.observable(m.flip_L(bDED=None, state='A'))
df['C8Flip_obs'] = result100.observable(m.C8(bf=None, flip=4, state='A') % m.flip_L(bDED=4, state='A'))
df['RIP1RIP3unmod_obs'] = result100.observable(m.RIP1(bscf=None, bub1=None, bub2=None, bub3=None, bDD=None, btraf=None, bRHIM=5, bMLKL=None, state='po4') % m.RIP3(bRHIM=5, bDD=1, state='po4') % m.MLKL(bRHIM=1, state='unmod'))
df['MLKLp_obs'] = result100.observable(m.MLKL(bRHIM=None, state='active'))



In [ ]:
len(df['MLKLp_obs'])

In [ ]:
tspan = np.linspace(0, 1440, 500)
x100 = np.array([30, 90, 270, 480, 600, 720, 840, 960])
y100 = np.array([0.00885691708746097,0.0161886154261265,0.0373005242261882,0.2798939020159581,0.510, .7797294067, 0.95,1])
plt.figure()
for i in range(len(mlkl)):
# plt.subplot(144)
    plt.plot(tspan/60, df['MLKLp_obs'])
# plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
# plt.scatter(x100/60, y100)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
# plt.legend()
plt.title(r'95% CI of pMLKL under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_25000params_TNF1_pydream_5chns_95CI.pdf', format='pdf')
plt.show()

In [ ]:
x100 = np.array([.5, 1.5, 4.5, 8, 12, 16])
y100 = np.array([
0.00885691708746097*5544,0.0161886154261265*5544,
0.0373005242261882*5544,
0.0798939020159581*5544,
0.639729406776*5544,
1*5544])
y10 = np.array([0.0106013664572332*5544,
0.00519576571714913*5544,
0.02967443048221*5544,
0.050022163974868*5544,
0.198128107774737*5544,
0.560551401148672*5544])
y1 = np.array([0.0060632926030143*5544,
0.00942691670200054*5544,
0.0113342231044983*5544,
0.0312716821493274*5544,
0.10956134602783*5544,
0.291058859808041*5544])
y0 = np.array([0.00777642331264696*5544,
0.00919829048298192*5544,
0.00177263616389778*5544,
0.00637961107042362*5544,
0.0136330127587403*5544,
0.10852337335697491*5544])

y100.std()

In [ ]:
time = result100.tout[0]

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# time = result100.tout[0]

# def mean_std(data,confidence=0.95):
#     a = 1.0 * np.array(data)
#     n = len(a)
#     m, se = np.mean(a), scipy.stats.sem(a)
#     std = np.std(a)
#     h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
#     return m, m-h, m+h, std

In [ ]:
print(len(time))

In [ ]:
idx = pd.MultiIndex.from_product([df.columns,
                                  ['mean', 'mean_minus', 'mean_plus']],
                                 names=['species', 'CI'])

ci_df = pd.DataFrame(0, idx, time)
# # ci_df.head()

# idx = pd.MultiIndex.from_product([df.columns,
#                                   ['mean', 'mean_minus', 'mean_plus', 'std']],
#                                  names=['species', 'CI'])

# ci_df = pd.DataFrame(0, idx, time)
# ci_df.head()

In [ ]:
ci_df.columns.values.shape

In [ ]:
from scipy import stats
for sp in df.columns:
    for t in time: 
        sp_values = df.xs(t, level='time', drop_level=False)[sp].values # axis=1 if columns
        ci_df.loc[[sp], t] = mean_confidence_interval(sp_values)

In [ ]:
p=500

In [ ]:
# mean = ci_df.loc['MLKLa_obs'][:1].values.reshape((1441,))
# upper = ci_df.loc['MLKLa_obs'][2:3].values.reshape((1441,))
# lower = ci_df.loc['MLKLa_obs'][1:2].values.reshape((1441,))
mean_pmlkl = ci_df.loc['MLKLa_obs'][:1].values.reshape((p,))
upper_pmlkl = ci_df.loc['MLKLa_obs'][2:3].values.reshape((p,))
lower_pmlkl = ci_df.loc['MLKLa_obs'][1:2].values.reshape((p,))
std_pmlkl = ci_df.loc['MLKLa_obs'][2:3].values.reshape((p,))
# mean_pmlkl = ci_df.loc['MLKLa_obs'][:1].values.reshape((9,))
# upper_pmlkl = ci_df.loc['MLKLa_obs'][2:3].values.reshape((9,))
# lower_pmlkl = ci_df.loc['MLKLa_obs'][1:2].values.reshape((9,))

In [ ]:
mean_flip = ci_df.loc['Flip_obs'][:1].values.reshape((p,))
upper_flip = ci_df.loc['Flip_obs'][2:3].values.reshape((p,))
lower_flip = ci_df.loc['Flip_obs'][1:2].values.reshape((p,))
# mean_flip = ci_df.loc['Flip_obs'][:1].values.reshape((9,))
# upper_flip = ci_df.loc['Flip_obs'][2:3].values.reshape((9,))
# lower_flip = ci_df.loc['Flip_obs'][1:2].values.reshape((9,))

In [ ]:
mean_C8flip = ci_df.loc['C8Flip_obs'][:1].values.reshape((p,))
upper_C8flip = ci_df.loc['C8Flip_obs'][2:3].values.reshape((p,))
lower_C8flip = ci_df.loc['C8Flip_obs'][1:2].values.reshape((p,))
# mean_C8flip = ci_df.loc['C8Flip_obs'][:1].values.reshape((9,))
# upper_C8flip = ci_df.loc['C8Flip_obs'][2:3].values.reshape((9,))
# lower_C8flip = ci_df.loc['C8Flip_obs'][1:2].values.reshape((9,))

In [ ]:
mean_necro = ci_df.loc['RIP1RIP3unmod_obs'][:1].values.reshape((p,))
upper_necro = ci_df.loc['RIP1RIP3unmod_obs'][2:3].values.reshape((p,))
lower_necro = ci_df.loc['RIP1RIP3unmod_obs'][1:2].values.reshape((p,))
# mean_necro = ci_df.loc['RIP1RIP3unmod_obs'][:1].values.reshape((9,))
# upper_necro = ci_df.loc['RIP1RIP3unmod_obs'][2:3].values.reshape((9,))
# lower_necro = ci_df.loc['RIP1RIP3unmod_obs'][1:2].values.reshape((9,))

In [ ]:
plt.figure(figsize = (20,5))
# plt.figure()
plt.subplot(141)
plt.plot(time/60, mean_flip, color = 'black', label ='mean', zorder = 2)
plt.fill_between(time/60, lower_flip, upper_flip, label ='95%CI',zorder = 1)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of cFlip-L under 100ng/ml of TNF$\alpha$')

# plt.figure()
plt.subplot(142)
plt.plot(time/60, mean_C8flip , color = 'black', label ='mean', zorder = 2)
plt.fill_between(time/60, lower_C8flip , upper_C8flip , label ='95%CI',zorder = 1)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of C8Flip 100ng/ml of TNF$\alpha$')

plt.subplot(143)
plt.plot(time/60, mean_necro, color = 'black', label ='mean', zorder = 2)
plt.fill_between(time/60, lower_necro, upper_necro, label ='95%CI',zorder = 1)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of Necrosome under 100ng/ml of TNF$\alpha$')

# plt.figure()
plt.subplot(144)
plt.plot(time/60, mean_pmlkl, color = 'black', label ='mean', zorder = 2)
plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of pMLKL under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_1000params_TNF100_pydream_5chns_95CI_C8FlipA20cIAPkd.pdf', format='pdf')
plt.show()

In [ ]:
plt.figure()
# plt.subplot(144)
plt.plot(time/60, mean_pmlkl, color = 'black', label ='mean', zorder = 2)
plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
# plt.errorbar(time/60, mean_pmlkl, std_pmlkl)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of pMLKL under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_25000params_TNF100_pydream_5chns_95CI_new100.pdf', format='pdf')
plt.show()

In [ ]:
print(result100.shape)

In [ ]:
# plt.figure(figsize = (20,5))
# # plt.figure()
# plt.subplot(141)
# plt.plot(time/60, mean_flip, color = 'black', label ='mean', zorder = 2)
# plt.fill_between(time/60, lower_flip, upper_flip, label ='95%CI',zorder = 1)
# plt.xlabel('Time (hours)')
# plt.ylabel('Amount (molecule/cell)')
# plt.legend()
# plt.title(r'95% CI of cFlip-L under 100ng/ml of TNF$\alpha$')

# # plt.figure()
# plt.subplot(142)
# plt.plot(time/60, mean_C8flip , color = 'black', label ='mean', zorder = 2)
# plt.fill_between(time/60, lower_C8flip , upper_C8flip , label ='95%CI',zorder = 1)
# plt.xlabel('Time (hours)')
# plt.ylabel('Amount (molecule/cell)')
# plt.legend()
# plt.title(r'95% CI of C8Flip 100ng/ml of TNF$\alpha$')

# plt.subplot(143)
# plt.plot(time/60, mean_necro, color = 'black', label ='mean', zorder = 2)
# plt.fill_between(time/60, lower_necro, upper_necro, label ='95%CI',zorder = 1)
# plt.xlabel('Time (hours)')
# plt.ylabel('Amount (molecule/cell)')
# plt.legend()
# plt.title(r'95% CI of Necrosome under 100ng/ml of TNF$\alpha$')

# plt.figure()
# plt.subplot(144)
plt.plot(time/60, result100.observable['MLKLa_obs'], color = 'black', label ='mean', zorder = 2)
# plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95%CI',zorder = 1)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title(r'95% CI of pMLKL under 100ng/ml of TNF$\alpha$')
plt.tight_layout()
# plt.savefig('necro_25000params_TNF1_pydream_5chns_95CI.pdf', format='pdf')
plt.show()

In [ ]:
ci_df.head()

In [ ]:
t = np.array([0., 30,  60,   120,  180, 270,  480,  960, 1440])
data = np.array([0.001, 0.003, 0.01, 0.0152, 0.03, 0.05, 0.5, 0.99, 1.])

In [ ]:
plt.figure()
plt.plot(time/60, mean_pmlkl, color = 'black', label ='mean', zorder = 3)
# plt.scatter(t/60, data*5544, color ='black', label = 'pMLKL_data',zorder = 2)
plt.fill_between(time/60, lower_pmlkl, upper_pmlkl, label ='95% CI',zorder = 1,alpha = 0.2)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title('95% Confidence Interval of pMLKL under 100ng/ml of TNFa')
plt.savefig('pMLKL_100TNF_95CI_CI.pdf', format = 'pdf')
plt.show()

In [ ]:
plt.figure()
plt.plot(time/60, mean_flip, color = 'black', label ='mean', zorder = 3)
plt.fill_between(time/60, lower_flip, upper_flip, label ='95% CI',zorder = 1, alpha = 0.2)
plt.xlabel('Time (hours)')
plt.ylabel('Amount (molecule/cell)')
plt.legend()
plt.title('95% Confidence Interval of pMLKL under 100ng/ml of TNFa')
# plt.savefig('pMLKL_100TNF_95CI_CI.pdf', format = 'pdf')
plt.show()

In [ ]:
# Visualize the result
plt.plot(time, mean, 'or')
# plt.plot(xfit, yfit, '-', color='gray')

plt.fill_between(time, lower, upper, color='gray', alpha=0.2)
plt.xlim(0, 10);

In [ ]:
# Define a function for the line plot with intervals
def lineplotCI(x_data, y_data, sorted_x, low_CI, upper_CI, x_label, y_label, title):
    # Create the plot object
    _, ax = plt.subplots()

    # Plot the data, set the linewidth, color and transparency of the
    # line, provide a label for the legend
    ax.plot(x_data, y_data, lw = 1, color = '#539caf', alpha = 1, label = 'Fit')
    # Shade the confidence interval
    ax.fill_between(sorted_x, low_CI, upper_CI, color = '#539caf', alpha = 0.4, label = '95% CI')
    # Label the axes and provide a title
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

    # Display legend
    ax.legend(loc = 'best')

In [ ]:
df = result100.dataframe.transpose()
df[0].ix['MLKLa_obs'][:]

In [ ]:
x = result100.observables
x[0]

In [ ]:
solver10 = ScipyOdeSimulator(model, tspan=tspan)
result10 = solver10.run(initials = {TNF(brec=None): 232}, param_values=all_pars[:])

In [ ]:
solver1 = ScipyOdeSimulator(model, tspan=tspan)
result1 = solver1.run(initials = {TNF(brec=None): 23}, param_values=all_pars[:])

In [ ]:
solver01 = ScipyOdeSimulator(model, tspan=tspan)
result01 = solver01.run(initials = {TNF(brec=None): 2}, param_values=all_pars[:])

In [ ]:
df = result100.dataframe(columns=['time', 'pmlkl'])
# df['time'] = len(df.loc[:]['MLKLa_obs'].iloc[:])
# df['pmlkl'] = df.loc[:]['MLKLa_obs'].iloc[:]
mlkl = df.loc[:]['MLKLa_obs'].iloc[:]
mlklt = mlkl.transpose()

In [ ]:
mlklt.head()

In [ ]:
mlkl.to_csv('pmlkl_10tpts.csv')

In [ ]:
df.head()

In [ ]:
a = df.loc[:]['CII_obs'].iloc[:]
a.head()

In [ ]:
b = a.T

In [ ]:
b.head()

In [ ]:
df.loc[:]['CII_C8a_obs'].iloc[:].max()

In [ ]:
CII_obs = df.loc[:]['CII_obs'].iloc[:]/df.loc[:]['CII_obs'].iloc[:].max()
CII_obs.head()

In [ ]:
df.loc[:]['CII_obs'].iloc[:].head()

In [ ]:
df.head()

In [ ]:
from tropical.visualize_trajectories import VisualizeTrajectories

In [ ]:
necro_clusters = VisualizeTrajectories(model, result100, clusters=None)

In [ ]:
necro_clusters.plot_cluster_dynamics(species=['MLKLa_obs'], fig_name='pmlkl_100ngml.png')
plt.show()

In [ ]:
necro_clusters.plot_cluster_dynamics(species=['CII_obs'], fig_name='CII_100ngml.png')
plt.show()

In [ ]:
necro_clusters.plot_cluster_dynamics(species=['CII_C8a_obs'], fig_name='CII_C8_100ngml.png')
plt.show()

In [ ]:
plt.figure()
for i in range(0,500):
    plt.plot(tspan, df.loc[:]['CII_obs'].iloc[:]/df.loc[:]['CII_obs'].iloc[:].max())
plt.show()

In [ ]:
pmlkl = df.loc[:]['MLKLa_obs'].iloc[:]
print(len(pmlkl))

In [ ]:
mean = []
mlklp = []
for i in range(len(tspan)):
    mlklp[i] = df.loc[i:i]['MLKLa_obs'].iloc[i:i]
    mean[i] = np.mean(mlklp[i]) 

In [ ]:
import pandas as pd
df_pmlkl = pd.DataFrame(columns=['time', 'pmlkl', 'meanpmlkl', 'lpmlkl', 'uplmlkl'])
df_pmlkl['time'] = len(df.loc[:]['MLKLa_obs'].iloc[:])
df_pmlkl['pmlkl'] = df.loc[:]['MLKLa_obs'].iloc[:]
df_pmlkl['meanpmlkl'] = np.mean(pmlkl)
df_pmlkl

In [ ]:
import numpy as np, scipy.stats as st
ci = st.t.interval(0.95, len(pmlkl)-1, loc=np.mean(pmlkl), scale=st.sem(pmlkl))

In [ ]:
import numpy as np
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
mean_confidence_interval(df.loc[:]['MLKLa_obs'].iloc[:], 0.95)

In [ ]:
# Filling between line y3 and line y4
plt.fill_between(tspan, 2392.356, 2402.73, color='grey', alpha='0.5')
plt.show()

In [ ]:
mean = np.mean(x) 
np.percentile(x,5)
np.percentile(x,95)

In [ ]:
ci

In [ ]:
ax = sns.lineplot(x="time", y="pMLKL", data=mlklp)